In [1]:
import numpy as np
import pandas as pd
import json
import ast
import os
import cv2

# Загрузка координат масок

In [2]:
data = pd.read_csv(r"C:\Users\Anton\Downloads\TlkWaterMeters\data.tsv", sep='\t', header=0)

In [3]:
data['location'] = data['location'].apply(lambda x: ast.literal_eval(x)['data'])

In [4]:
# берём картинки у которых маска -- чытерёхугольник
data = data[data['location'].map(len) == 4].reset_index(drop=True)

In [6]:
pd.set_option('max_colwidth', 400)
data

,photo_name,value,location
0,id_407_value_21_86.jpg,21.860,"[{'x': 0.27545, 'y': 0.19134}, {'x': 0.37483, 'y': 0.18282}, {'x': 0.38935, 'y': 0.76071}, {'x': 0.28185, 'y': 0.76613}]"
1,id_252_value_313_322.jpg,313.322,"[{'x': 0.21967, 'y': 0.50845}, {'x': 0.70811, 'y': 0.53256}, {'x': 0.70008, 'y': 0.61295}, {'x': 0.21699, 'y': 0.58883}]"
2,id_1204_value_78_677.jpg,78.677,"[{'x': 0.26854, 'y': 0.38812}, {'x': 0.56977, 'y': 0.33109}, {'x': 0.57225, 'y': 0.41451}, {'x': 0.27392, 'y': 0.47409}]"
3,id_672_value_350_806.jpg,350.806,"[{'x': 0.36175, 'y': 0.59318}, {'x': 0.35917, 'y': 0.6288}, {'x': 0.59294, 'y': 0.63693}, {'x': 0.59448, 'y': 0.60093}]"
4,id_2_value_495_341.jpg,495.341,"[{'x': 0.26521, 'y': 0.5183}, {'x': 0.81616, 'y': 0.51579}, {'x': 0.81973, 'y': 0.57957}, {'x': 0.27057, 'y': 0.58409}]"
...,...,...,...
500,id_1146_value_90_393.jpg,90.393,"[{'x': 0.24256, 'y': 0.38728}, {'x': 0.74151, 'y': 0.41624}, {'x': 0.72884, 'y': 0.50441}, {'x': 0.23795, 'y': 0.47934}]"
501,id_700_value_77_542.jpg,77.542,"[{'x': 0.2531, 'y': 0.49937}, {'x': 0.25536, 'y': 0.46072}, {'x': 0.63144, 'y': 0.48273}, {'x': 0.62655, 'y': 0.52166}]"
502,id_255_value_172_542.jpg,172.542,"[{'x': 0.2368, 'y': 0.48755}, {'x': 0.74324, 'y': 0.47545}, {'x': 0.74439, 'y': 0.56103}, {'x': 0.23968, 'y': 0.57357}]"
503,id_878_value_97_299.jpg,97.299,"[{'x': 0.30909, 'y': 0.55625}, {'x': 0.66163, 'y': 0.54632}, {'x': 0.6637, 'y': 0.60188}, {'x': 0.31447, 'y': 0.61027}]"


In [35]:
def sort_coords(coords):
    by_x = sorted(coords, key=lambda coord: [coord['x'], coord['y']])
    left = sorted(by_x[:2], key=lambda coord: [coord['y'], coord['x']])
    right = sorted(by_x[2:], key=lambda coord: [coord['y'], coord['x']])
    
    return left + right

In [36]:
coordinates_df = pd.DataFrame(data['location'].apply(sort_coords).to_list(), columns = ['bottom_left', 
                                                                                        'upper_left', 
                                                                                        'bottom_right', 
                                                                                        'upper_right'])
coordinates_df

,bottom_left,upper_left,bottom_right,upper_right
0,"{'x': 0.27545, 'y': 0.19134}","{'x': 0.28185, 'y': 0.76613}","{'x': 0.37483, 'y': 0.18282}","{'x': 0.38935, 'y': 0.76071}"
1,"{'x': 0.21967, 'y': 0.50845}","{'x': 0.21699, 'y': 0.58883}","{'x': 0.70811, 'y': 0.53256}","{'x': 0.70008, 'y': 0.61295}"
2,"{'x': 0.26854, 'y': 0.38812}","{'x': 0.27392, 'y': 0.47409}","{'x': 0.56977, 'y': 0.33109}","{'x': 0.57225, 'y': 0.41451}"
3,"{'x': 0.36175, 'y': 0.59318}","{'x': 0.35917, 'y': 0.6288}","{'x': 0.59448, 'y': 0.60093}","{'x': 0.59294, 'y': 0.63693}"
4,"{'x': 0.26521, 'y': 0.5183}","{'x': 0.27057, 'y': 0.58409}","{'x': 0.81616, 'y': 0.51579}","{'x': 0.81973, 'y': 0.57957}"
...,...,...,...,...
500,"{'x': 0.24256, 'y': 0.38728}","{'x': 0.23795, 'y': 0.47934}","{'x': 0.74151, 'y': 0.41624}","{'x': 0.72884, 'y': 0.50441}"
501,"{'x': 0.25536, 'y': 0.46072}","{'x': 0.2531, 'y': 0.49937}","{'x': 0.63144, 'y': 0.48273}","{'x': 0.62655, 'y': 0.52166}"
502,"{'x': 0.2368, 'y': 0.48755}","{'x': 0.23968, 'y': 0.57357}","{'x': 0.74324, 'y': 0.47545}","{'x': 0.74439, 'y': 0.56103}"
503,"{'x': 0.30909, 'y': 0.55625}","{'x': 0.31447, 'y': 0.61027}","{'x': 0.66163, 'y': 0.54632}","{'x': 0.6637, 'y': 0.60188}"


In [37]:
data_coords = pd.concat([data.drop(columns=['location']), coordinates_df], axis=1)
data_coords

,photo_name,value,bottom_left,upper_left,bottom_right,upper_right
0,id_407_value_21_86.jpg,21.860,"{'x': 0.27545, 'y': 0.19134}","{'x': 0.28185, 'y': 0.76613}","{'x': 0.37483, 'y': 0.18282}","{'x': 0.38935, 'y': 0.76071}"
1,id_252_value_313_322.jpg,313.322,"{'x': 0.21967, 'y': 0.50845}","{'x': 0.21699, 'y': 0.58883}","{'x': 0.70811, 'y': 0.53256}","{'x': 0.70008, 'y': 0.61295}"
2,id_1204_value_78_677.jpg,78.677,"{'x': 0.26854, 'y': 0.38812}","{'x': 0.27392, 'y': 0.47409}","{'x': 0.56977, 'y': 0.33109}","{'x': 0.57225, 'y': 0.41451}"
3,id_672_value_350_806.jpg,350.806,"{'x': 0.36175, 'y': 0.59318}","{'x': 0.35917, 'y': 0.6288}","{'x': 0.59448, 'y': 0.60093}","{'x': 0.59294, 'y': 0.63693}"
4,id_2_value_495_341.jpg,495.341,"{'x': 0.26521, 'y': 0.5183}","{'x': 0.27057, 'y': 0.58409}","{'x': 0.81616, 'y': 0.51579}","{'x': 0.81973, 'y': 0.57957}"
...,...,...,...,...,...,...
500,id_1146_value_90_393.jpg,90.393,"{'x': 0.24256, 'y': 0.38728}","{'x': 0.23795, 'y': 0.47934}","{'x': 0.74151, 'y': 0.41624}","{'x': 0.72884, 'y': 0.50441}"
501,id_700_value_77_542.jpg,77.542,"{'x': 0.25536, 'y': 0.46072}","{'x': 0.2531, 'y': 0.49937}","{'x': 0.63144, 'y': 0.48273}","{'x': 0.62655, 'y': 0.52166}"
502,id_255_value_172_542.jpg,172.542,"{'x': 0.2368, 'y': 0.48755}","{'x': 0.23968, 'y': 0.57357}","{'x': 0.74324, 'y': 0.47545}","{'x': 0.74439, 'y': 0.56103}"
503,id_878_value_97_299.jpg,97.299,"{'x': 0.30909, 'y': 0.55625}","{'x': 0.31447, 'y': 0.61027}","{'x': 0.66163, 'y': 0.54632}","{'x': 0.6637, 'y': 0.60188}"


In [38]:
data_coords['coordinates'] = data_coords.apply(lambda x: list(x['bottom_left'].values()) + \
                                               list(x['upper_left'].values()) + \
                                               list(x['upper_right'].values()) + \
                                               list(x['bottom_right'].values()), axis=1)
data_coords['coordinates'] = data_coords['coordinates'].apply(lambda x: np.array(x, dtype='float'))
data_coords

,photo_name,value,bottom_left,upper_left,bottom_right,upper_right,coordinates
0,id_407_value_21_86.jpg,21.860,"{'x': 0.27545, 'y': 0.19134}","{'x': 0.28185, 'y': 0.76613}","{'x': 0.37483, 'y': 0.18282}","{'x': 0.38935, 'y': 0.76071}","[0.27545, 0.19134, 0.28185, 0.76613, 0.38935, ..."
1,id_252_value_313_322.jpg,313.322,"{'x': 0.21967, 'y': 0.50845}","{'x': 0.21699, 'y': 0.58883}","{'x': 0.70811, 'y': 0.53256}","{'x': 0.70008, 'y': 0.61295}","[0.21967, 0.50845, 0.21699, 0.58883, 0.70008, ..."
2,id_1204_value_78_677.jpg,78.677,"{'x': 0.26854, 'y': 0.38812}","{'x': 0.27392, 'y': 0.47409}","{'x': 0.56977, 'y': 0.33109}","{'x': 0.57225, 'y': 0.41451}","[0.26854, 0.38812, 0.27392, 0.47409, 0.57225, ..."
3,id_672_value_350_806.jpg,350.806,"{'x': 0.36175, 'y': 0.59318}","{'x': 0.35917, 'y': 0.6288}","{'x': 0.59448, 'y': 0.60093}","{'x': 0.59294, 'y': 0.63693}","[0.36175, 0.59318, 0.35917, 0.6288, 0.59294, 0..."
4,id_2_value_495_341.jpg,495.341,"{'x': 0.26521, 'y': 0.5183}","{'x': 0.27057, 'y': 0.58409}","{'x': 0.81616, 'y': 0.51579}","{'x': 0.81973, 'y': 0.57957}","[0.26521, 0.5183, 0.27057, 0.58409, 0.81973, 0..."
...,...,...,...,...,...,...,...
500,id_1146_value_90_393.jpg,90.393,"{'x': 0.24256, 'y': 0.38728}","{'x': 0.23795, 'y': 0.47934}","{'x': 0.74151, 'y': 0.41624}","{'x': 0.72884, 'y': 0.50441}","[0.24256, 0.38728, 0.23795, 0.47934, 0.72884, ..."
501,id_700_value_77_542.jpg,77.542,"{'x': 0.25536, 'y': 0.46072}","{'x': 0.2531, 'y': 0.49937}","{'x': 0.63144, 'y': 0.48273}","{'x': 0.62655, 'y': 0.52166}","[0.25536, 0.46072, 0.2531, 0.49937, 0.62655, 0..."
502,id_255_value_172_542.jpg,172.542,"{'x': 0.2368, 'y': 0.48755}","{'x': 0.23968, 'y': 0.57357}","{'x': 0.74324, 'y': 0.47545}","{'x': 0.74439, 'y': 0.56103}","[0.2368, 0.48755, 0.23968, 0.57357, 0.74439, 0..."
503,id_878_value_97_299.jpg,97.299,"{'x': 0.30909, 'y': 0.55625}","{'x': 0.31447, 'y': 0.61027}","{'x': 0.66163, 'y': 0.54632}","{'x': 0.6637, 'y': 0.60188}","[0.30909, 0.55625, 0.31447, 0.61027, 0.6637, 0..."


# Загрузка изображений

In [10]:
images_data = []
dirname = r"C:\Users\Anton\Downloads\TlkWaterMeters\images"

for filename in data_coords['photo_name']:
    img = cv2.imread(os.path.join(dirname, filename)) / 255.0
    img = cv2.resize(img, (224, 224))
    img = img.flatten()
    images_data.append(img)

images_data = np.array(images_data)

In [11]:
gray_images_data = []
dirname = r"C:\Users\Anton\Downloads\TlkWaterMeters\images"

for filename in data_coords['photo_name']:
    img = cv2.imread(os.path.join(dirname, filename))
    img = cv2.resize(img, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     (thresh, img) = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
    img = img.flatten()
    gray_images_data.append(img)

gray_images_data = np.array(gray_images_data)

In [12]:
bw_images_data = []
dirname = r"C:\Users\Anton\Downloads\TlkWaterMeters\images"

for filename in data_coords['photo_name']:
    img = cv2.imread(os.path.join(dirname, filename))
    img = cv2.resize(img, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    (thresh, img) = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
    img = img.flatten()
    bw_images_data.append(img)

bw_images_data = np.array(bw_images_data)

In [13]:
masks_data = []
dirname = r"C:\Users\Anton\Downloads\TlkWaterMeters\masks"

for filename in data_coords['photo_name']:
    img = cv2.imread(os.path.join(dirname, filename)) / 255.0
    img = cv2.resize(img, (224, 224))
    img = img.flatten()
    masks_data.append(img)

masks_data = np.array(masks_data)

# Линейная регрессия

In [39]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [40]:
x_train, x_test, y_train, y_test = train_test_split(images_data, 
                                                    np.array(data_coords['coordinates'].values.tolist()), 
                                                    test_size=0.33, random_state=42)

In [41]:
lr = LinearRegression()
lr.fit(x_train, y_train)

LinearRegression()

In [42]:
y_pred = lr.predict(x_test)

In [54]:
from shapely.geometry import Polygon

In [99]:
def intersection_loss(y_true, y_pred):
    test = y_true.reshape((4, 2))
    pred = y_pred.reshape((4, 2))
    
    try:
        poly_pred = Polygon(pred)
        poly_test = Polygon(test)

        intersection_area = poly_pred.intersection(poly_test).area
    except:
        return None
    
    return intersection_area/poly_test.area

In [64]:
np.array([intersection_loss(*y) for y in zip(y_test, y_pred)]).mean()

0.23126000166901448

# Линейная регрессия с grayscale изображениями

In [65]:
gray_x_train, gray_x_test, gray_y_train, gray_y_test = train_test_split(gray_images_data, 
                                                                        np.array(data_coords['coordinates'].values.tolist()), 
                                                                        test_size=0.33, random_state=42)

In [66]:
gray_lr = LinearRegression()
gray_lr.fit(gray_x_train, gray_y_train)

LinearRegression()

In [67]:
gray_y_pred = gray_lr.predict(gray_x_test)

In [68]:
np.array([intersection_loss(*y) for y in zip(gray_y_test, gray_y_pred)]).mean()

0.2248381983487344

# Линейная регрессия с черно-белыми изображениями

In [69]:
bw_x_train, bw_x_test, bw_y_train, bw_y_test = train_test_split(bw_images_data, 
                                                                np.array(data_coords['coordinates'].values.tolist()), 
                                                                test_size=0.33, random_state=42)

In [70]:
bw_lr = LinearRegression()
bw_lr.fit(bw_x_train, bw_y_train)

LinearRegression()

In [71]:
bw_y_pred = bw_lr.predict(bw_x_test)

In [72]:
np.array([intersection_loss(*y) for y in zip(bw_y_test, bw_y_pred)]).mean()

0.22494914587975845

# KNeighborsRegressor + grayscale

In [77]:
from sklearn.neighbors import KNeighborsRegressor

In [78]:
parameters = {
    'n_neighbors': [3, 5, 10],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}

In [76]:
models = []
best_score = 0
best_params = {}
best_model = None

for n_neighbors in parameters['n_neighbors']:
    for weights in parameters['weights']:
        for p in parameters['p']:
            print(f'START: n_neighbors={n_neighbors}, weights={weights}, p={p}', end='... ')
            
            model = KNeighborsRegressor(n_neighbors=n_neighbors, weights=weights, p=p)
            model.fit(gray_x_train, gray_y_train)
            models.append(model)
            
            y_pred_knn = model.predict(gray_x_test)
            score = np.array([intersection_loss(*y) for y in zip(gray_y_test, y_pred_knn)]).mean()
            print(f'SCORE: {score:.3f}', end='... ')
            
            if score > best_score:
                best_model = model
                best_score = score
                best_params['n_neighbors'] = n_neighbors
                best_params['weights'] = weights
                best_params['p'] = p
            
            print('END')

START: n_neighbors=3, weights=uniform, p=1... SCORE: 0.247... END
START: n_neighbors=3, weights=uniform, p=2... SCORE: 0.272... END
START: n_neighbors=3, weights=distance, p=1... SCORE: 0.249... END
START: n_neighbors=3, weights=distance, p=2... SCORE: 0.273... END
START: n_neighbors=5, weights=uniform, p=1... SCORE: 0.239... END
START: n_neighbors=5, weights=uniform, p=2... SCORE: 0.252... END
START: n_neighbors=5, weights=distance, p=1... SCORE: 0.240... END
START: n_neighbors=5, weights=distance, p=2... SCORE: 0.254... END
START: n_neighbors=10, weights=uniform, p=1... SCORE: 0.229... END
START: n_neighbors=10, weights=uniform, p=2... SCORE: 0.248... END
START: n_neighbors=10, weights=distance, p=1... SCORE: 0.227... END
START: n_neighbors=10, weights=distance, p=2... SCORE: 0.249... END


# KNeighborsRegressor

In [79]:
models = []
best_score = 0
best_params = {}
best_model = None

for n_neighbors in parameters['n_neighbors']:
    for weights in parameters['weights']:
        for p in parameters['p']:
            print(f'START: n_neighbors={n_neighbors}, weights={weights}, p={p}', end='... ')
            
            model = KNeighborsRegressor(n_neighbors=n_neighbors, weights=weights, p=p)
            model.fit(x_train, y_train)
            models.append(model)
            
            y_pred_knn = model.predict(x_test)
            score = np.array([intersection_loss(*y) for y in zip(y_test, y_pred_knn)]).mean()
            print(f'SCORE: {score:.3f}', end='... ')
            
            if score > best_score:
                best_model = model
                best_score = score
                best_params['n_neighbors'] = n_neighbors
                best_params['weights'] = weights
                best_params['p'] = p
            
            print('END')

START: n_neighbors=3, weights=uniform, p=1... SCORE: 0.246... END
START: n_neighbors=3, weights=uniform, p=2... SCORE: 0.309... END
START: n_neighbors=3, weights=distance, p=1... SCORE: 0.247... END
START: n_neighbors=3, weights=distance, p=2... SCORE: 0.309... END
START: n_neighbors=5, weights=uniform, p=1... SCORE: 0.246... END
START: n_neighbors=5, weights=uniform, p=2... SCORE: 0.275... END
START: n_neighbors=5, weights=distance, p=1... SCORE: 0.247... END
START: n_neighbors=5, weights=distance, p=2... SCORE: 0.275... END
START: n_neighbors=10, weights=uniform, p=1... SCORE: 0.253... END
START: n_neighbors=10, weights=uniform, p=2... SCORE: 0.276... END
START: n_neighbors=10, weights=distance, p=1... SCORE: 0.253... END
START: n_neighbors=10, weights=distance, p=2... SCORE: 0.277... END


# KNeighborsRegressor + Black&White

In [80]:
models = []
best_score = 0
best_params = {}
best_model = None

for n_neighbors in parameters['n_neighbors']:
    for weights in parameters['weights']:
        for p in parameters['p']:
            print(f'START: n_neighbors={n_neighbors}, weights={weights}, p={p}', end='... ')
            
            model = KNeighborsRegressor(n_neighbors=n_neighbors, weights=weights, p=p)
            model.fit(bw_x_train, bw_y_train)
            models.append(model)
            
            y_pred_knn = model.predict(bw_x_test)
            score = np.array([intersection_loss(*y) for y in zip(bw_y_test, y_pred_knn)]).mean()
            print(f'SCORE: {score:.3f}', end='... ')
            
            if score > best_score:
                best_model = model
                best_score = score
                best_params['n_neighbors'] = n_neighbors
                best_params['weights'] = weights
                best_params['p'] = p
            
            print('END')

START: n_neighbors=3, weights=uniform, p=1... SCORE: 0.218... END
START: n_neighbors=3, weights=uniform, p=2... SCORE: 0.218... END
START: n_neighbors=3, weights=distance, p=1... SCORE: 0.222... END
START: n_neighbors=3, weights=distance, p=2... SCORE: 0.220... END
START: n_neighbors=5, weights=uniform, p=1... SCORE: 0.214... END
START: n_neighbors=5, weights=uniform, p=2... SCORE: 0.214... END
START: n_neighbors=5, weights=distance, p=1... SCORE: 0.216... END
START: n_neighbors=5, weights=distance, p=2... SCORE: 0.215... END
START: n_neighbors=10, weights=uniform, p=1... SCORE: 0.194... END
START: n_neighbors=10, weights=uniform, p=2... SCORE: 0.194... END
START: n_neighbors=10, weights=distance, p=1... SCORE: 0.190... END
START: n_neighbors=10, weights=distance, p=2... SCORE: 0.189... END


# DecisionTreeClassifier

In [81]:
from sklearn.tree import DecisionTreeRegressor

In [88]:
parameters = {
    'max_depth': [20, 40, 100],
}

In [89]:
for max_depth in parameters['max_depth']:
        print(f'START: max_depth={max_depth}', end='... ')

        model = DecisionTreeRegressor(criterion=criterion, max_depth=max_depth)
        model.fit(x_train, y_train)
        models.append(model)

        y_pred_knn = model.predict(x_test)
        score = np.array([intersection_loss(*y) for y in zip(y_test, y_pred_knn)]).mean()
        print(f'SCORE: {score:.3f}', end='... ')
        print('END')

START: max_depth=20... SCORE: 0.171... END
START: max_depth=40... SCORE: 0.194... END
START: max_depth=100... SCORE: 0.159... END


# MLPRegressor 

In [90]:
from sklearn.neural_network import MLPRegressor

In [103]:
mlpr = MLPRegressor(hidden_layer_sizes=(100,50),
                    activation = 'relu',
                    solver='adam',
                    verbose=True)

In [104]:
mlpr.fit(x_train, y_train)

Iteration 1, loss = 1395.97725365
Iteration 2, loss = 193.63373786
Iteration 3, loss = 39.57384535
Iteration 4, loss = 31.74408005
Iteration 5, loss = 29.29883846
Iteration 6, loss = 11.61366080


C:\Users\Anton\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:709: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPRegressor(hidden_layer_sizes=(100, 50), verbose=True)

In [97]:
y_pred = mlpr.predict(x_test)

In [101]:
a = np.array([intersection_loss(*y) for y in zip(y_test, y_pred)])
a[a != np.array(None)].mean()

TopologyException: Input geom 0 is invalid: Self-intersection at 0.47592797071999532 0.65946387464772172
TopologyException: Input geom 0 is invalid: Self-intersection at 0.414084625074917 0.43122817037300354
TopologyException: Input geom 0 is invalid: Self-intersection at 0.54872812326739628 0.43599364057091983
TopologyException: Input geom 0 is invalid: Self-intersection at 0.67087299275530665 0.54186017319884272
TopologyException: Input geom 0 is invalid: Self-intersection at 0.61703149627613896 0.78224553520823203
TopologyException: Input geom 0 is invalid: Self-intersection at 0.32465266524276937 0.32229281558957507
TopologyException: Input geom 0 is invalid: Self-intersection at 0.47472652623474115 0.46591341701448363
TopologyException: Input geom 0 is invalid: Self-intersection at 0.53654247003525102 0.49835146753209075
TopologyException: Input geom 0 is invalid: Self-intersection at 0.65679772858304364 0.61745423046435699
TopologyException: Input geom 0 is invalid: Self-intersec

0.19705300094208666